In [1]:
#Importing the Dataset
import pandas as pd
messages=pd.read_csv('Rating_prediction.csv')

In [2]:
messages["Rating"].replace({"4.2 out of 5":"4","4 out of 5":"4","4.1 out of 5":"4","3.8 out of 5":"4","3.7 out of 5":"4","4.3 out of 5":"4","3.9 out of 5":"4","4.4 out of 5":"4","3.2 out of 5":"3","3.6 out of 5":"4","5 out of 5":"5","3.3 out of 5":"3","3.5 out of 5":"4","3.4 out of 5":"3","4.7 out of 5":"5","3.1 out of 5":"3","1 out of 5":"1","3 out of 5":"3","2.8 out of 5":"3","4.5 out of 5":"5","2.7 out of 5":"3","2.5 out of 5":"3","4.6 out of 5":"5"},inplace=True)
messages.head(5)

,Unnamed: 0,Unnamed: 0.1,Review,Rating
0,0,0.0,Received this yesterday (04/03/2021). Prompt d...,4
1,1,1.0,-,-
2,2,2.0,I bought this laptop with 8gb of ram and 256GB...,4
3,3,3.0,The display cracked inside after 2 months and ...,4
4,4,4.0,Laptop getting hang very much and very much sl...,4


In [3]:
#Drop "Unnamed:0" as it is only nominal data
messages=messages.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
messages.head()

,Review,Rating
0,Received this yesterday (04/03/2021). Prompt d...,4
1,-,-
2,I bought this laptop with 8gb of ram and 256GB...,4
3,The display cracked inside after 2 months and ...,4
4,Laptop getting hang very much and very much sl...,4


In [4]:
#Data Cleaning and Preprocessing
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
messages['Review']=messages['Review'].apply(str)
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['Review'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\welcome\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#Creating the Bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)

x=cv.fit_transform(corpus).toarray()

y=pd.get_dummies(messages['Rating'])
y=y.iloc[ : ,1].values

In [6]:
#Train Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [7]:
#Training model using Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
Rating_prediction_model=MultinomialNB().fit(x_train,y_train)

y_pred=Rating_prediction_model.predict(x_test)

In [8]:
#Comparing using Confusion matrix
from sklearn.metrics import confusion_matrix
confusion_m=confusion_matrix(y_test,y_pred)

In [9]:
#Checking Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [10]:
accuracy

0.9977678571428571

.Model is 99% fit the data.

In [11]:
#Saving the model
import pickle
Rating_project='finalized_model.pickle'
pickle.dump(Rating_prediction_model,open(Rating_project,'wb'))